In [ ]:
from collections import deque
import queue
import copy

In [ ]:
class graph():
    def __init__(self, nodes, connections, connectionsIds):
        self.connections = connections
        self.nodes = nodes
        self.connectionsIds = connectionsIds
        
    def DFSUtil(self, v, visited):
 
        visited.add(v)
        print(v, end=' ')

        for neighbour in self.connections[v]:
            if neighbour.nodeId not in visited:
                self.DFSUtil(neighbour.nodeId, visited)


    def DFS(self, v):

        visited = set()
        self.DFSUtil(v, visited)

In [ ]:
class node():
    def __init__(self, nodeId = None, nodeDepth = None, 
                 domain = None, parent = None, notAssignedVars = None, color = None):
        
        self.nodeId = nodeId
        self.neighbors = []
        self.nodeDepth = nodeDepth
        self.domain = domain
        self.parent = parent
        self.notAssignedVars = notAssignedVars
        self.color = color

In [ ]:
def printBackwards(node):
    while node.parent:
        print("NODE ID : " + str(node.nodeId) + " - Node Color: " + str(node.color))
        node = node.parent

In [ ]:
def readInputTextFile(file):
    connections = {}
    connectionsIds = {}
    nodes = {}
    while True:
        line = file.readline().rstrip('\n')
        if line == "":
            break
        elif(line[0] == "#"):
            continue
        elif(line[:6].lower() == "colors"):
            colors = int(line.split("=")[1])
            
        else:
            s, d = [int(i) for i in line.split(",")]
            if s in connections:
                connections[s].append(node(nodeId = d))
                connectionsIds[s].append(d)
                
                if d in connections:
                    connections[d].append(node(nodeId = s))
                    connectionsIds[d].append(s)
                else:
                    connections[d] = []
                    connectionsIds[d] = []
                    connections[d].append(node(nodeId = s))
                    connectionsIds[d].append(s)
                    nodes[d] = node(nodeId=d)
                    
                    
            else:
                connections[s] = []
                connectionsIds[s] = []
                connections[s].append(node(nodeId = d))
                connectionsIds[s].append(d)
                nodes[s] = node(nodeId = s)
                
                if d in connections:
                    connections[d].append(node(nodeId = s))
                    connectionsIds[d].append(s)
                else:
                    connections[d] = []
                    connectionsIds[d] = []
                    connections[d].append(node(nodeId = s))
                    connectionsIds[d].append(s)
                    nodes[d] = node(nodeId = d)

                
    return colors, nodes, connections, connectionsIds
        

In [ ]:
def minRemainingVal(node):
    mrv = 2**100
    mrv_var = None
    
    for variable in node.notAssignedVars:
        if len(node.domain[variable]) < mrv and len(node.domain[variable]) != 0:
            mrv = len(node.domain[variable])
            mrv_var = variable
    
    return mrv_var

In [ ]:
def AC3(domain):
    arcs = []
    
    for var in list(g.connectionsIds.keys()):
        for var2 in g.connectionsIds[var]:
            arcs.append([var, var2])
    
    
    while arcs:
        arcVars = arcs.pop(0)
        if revise(arcVars[0], arcVars[1], domain):
        
            neighbors = copy.deepcopy(g.connectionsIds[arcVars[0]])
            neighbors.remove(arcVars[1])

            for xk in neighbors:
                arcs.append([xk, arcVars[0]])
    
    return True

def revise( Xi, Xj, domain):
    revised = False

    #for each value in the domain
    for x in domain[Xi]:
        for j in domain[Xj]:
            if ( x != j):
                break
            else:
                domain[Xi].remove(x)
                revised = True
            
        if(len(domain[Xi]) == 0):
            sys.exit("No Solutions Exists")
            
    return revised

In [ ]:
def graphColor(MRV = True, LCV = True, ArcCns = True):
    
    #initial Domain 

    domain = {}

    for nodeId in list(g.connections.keys()):
        domain[nodeId] = []
        for color in [c for c in range(colors)]:
            domain[nodeId].append(color)

    degrees = {}
    for nodeId in list(g.connections.keys()):
        degrees[nodeId] = len(g.connections[nodeId])
    
    if(ArcCns):
        AC3(domain)
    
    print("***** Initial Domain *****")
    print(domain)
    print("\n")
    
    rootNodeId = list(g.connections.keys())[0]
    nodeIdsInGraph = list(g.connections.keys())

    rootNode = node(nodeId = -1, nodeDepth = 0, domain = domain, notAssignedVars = nodeIdsInGraph)

    fringe = deque()
    fringe.append(rootNode)

    targetDepth = len(g.connections)
    
    while (len(fringe) > 0):

        currNode = fringe.popleft()

        if currNode.nodeDepth == targetDepth:
            print("Graph colored successfully")
            return currNode
        
        if(MRV):
            nodeIdColor = minRemainingVal(currNode)
            
        else:
            try:
                nodeIdColor = currNode.notAssignedVars[0]
            except:
                continue
                
        if(nodeIdColor == None):
                continue
        currNode.notAssignedVars.remove(nodeIdColor)

        nodesLCV = []
        counter = 0
        for color in currNode.domain[nodeIdColor]:
            constCount = 0


            tempDomain = copy.deepcopy(currNode.domain)
            tempDomain[nodeIdColor].remove(color)

            for neighbor in g.connections[nodeIdColor]:
                if color in tempDomain[neighbor.nodeId]:
                    constCount += 1
                    tempDomain[neighbor.nodeId].remove(color)


            newNode = node(nodeId = nodeIdColor, 
                                  nodeDepth = currNode.nodeDepth+1, 
                                  domain = tempDomain,
                                  notAssignedVars = currNode.notAssignedVars,
                                  color = color,
                                  parent = currNode)

            if( not LCV):
                fringe.appendleft(newNode)
            else:
                nodesLCV.append((constCount,counter, newNode))
                counter += 1

        if(LCV):
            for n in sorted(nodesLCV, reverse = True):
                fringe.appendleft(n[2])
    
    print("NO SOLUTION FOUND")
    return(currNode)

In [ ]:
file = open("input.txt", "r")
colors, nodes, connections, connectionsIds = readInputTextFile(file)
g = graph(nodes =nodes, connections = connections, connectionsIds = connectionsIds)

In [ ]:
g.DFS(1)

In [ ]:
solution = graphColor(ArcCns=True, MRV=True, LCV=True)

In [ ]:
printBackwards(solution)